#Importing required files

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight

In [ ]:
train_df=pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv',header=None)
test_df=pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv',header=None)

In [ ]:
train_df[187]=train_df[187].astype(int)
equilibre=train_df[187].value_counts()
print(equilibre)

#Data Visualization

In [ ]:
x=np.arange(0,187)*0.008
plt.figure(figsize=(10,5))
plt.plot(x,X[C0,:][0],label="Cat N")
plt.legend()
plt.title('ECG for C0',fontsize=16)
plt.ylabel('Amplitude',fontsize=10)
plt.xlabel('Time',fontsize=10)
plt.show()

In [ ]:
x=np.arange(0,187)*0.008
plt.figure(figsize=(10,5))
plt.plot(x,X[C1,:][0],label="Cat S")
plt.legend()
plt.title('ECG for C1',fontsize=16)
plt.ylabel('Amplitude',fontsize=10)
plt.xlabel('Time',fontsize=10)
plt.show()

In [ ]:
x=np.arange(0,187)*0.008
plt.figure(figsize=(10,5))
plt.plot(x,X[C2,:][0],label="Cat V")
plt.legend()
plt.title('ECG for C2',fontsize=16)
plt.ylabel('Amplitude',fontsize=10)
plt.xlabel('Time',fontsize=10)
plt.show()

In [ ]:
x=np.arange(0,187)*0.008
plt.figure(figsize=(10,5))
plt.plot(x,X[C3,:][0],label="Cat F")
plt.legend()
plt.title('ECG for C3',fontsize=16)
plt.ylabel('Amplitude',fontsize=10)
plt.xlabel('Time',fontsize=10)
plt.show()

In [ ]:
x=np.arange(0,187)*0.008
plt.figure(figsize=(10,5))
plt.plot(x,X[C4,:][0],label="Cat Q")
plt.legend()
plt.title('ECG for C4',fontsize=16)
plt.ylabel('Amplitude',fontsize=10)
plt.xlabel('Time',fontsize=10)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(x,X[C0,:][0],label="Cat N")
plt.plot(x,X[C1,:][0],label="Cat S")
plt.plot(x,X[C2,:][0],label="Cat V")
plt.plot(x,X[C3,:][0],label="Cat F")
plt.plot(x,X[C4,:][0],label="Cat Q")
plt.legend()
plt.title('1 beat for each category',fontsize=16)
plt.ylabel('Amplitude',fontsize=10)
plt.xlabel('Time',fontsize=10)
plt.show()

#Model creation

In [ ]:
target_train=train_df[187]
target_test=test_df[187]
y_train=to_categorical(target_train)
y_test=to_categorical(target_test)


In [ ]:
_train=train_df.iloc[:,:186].values
X_test=test_df.iloc[:,:186].values
#for i in range(len(X_train)):
#    X_train[i,:186]= add_gaussian_noise(X_train[i,:186])
X_train = X_train.reshape(len(X_train), X_train.shape[1],1)
X_test = X_test.reshape(len(X_test), X_test.shape[1],1)

#3 layer CNN Model

In [ ]:
def network(X_train,y_train,X_test,y_test):
    

    im_shape=(X_train.shape[1],1)
    inputs_cnn=Input(shape=(im_shape), name='inputs_cnn')
    conv1_1=Convolution1D(64, (6), activation='relu', input_shape=im_shape)(inputs_cnn)
    conv1_1=BatchNormalization()(conv1_1)
    pool1=MaxPool1D(pool_size=(3), strides=(2), padding="same")(conv1_1)
    conv2_1=Convolution1D(64, (3), activation='relu', input_shape=im_shape)(pool1)
    conv2_1=BatchNormalization()(conv2_1)
    pool2=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv2_1)
    conv3_1=Convolution1D(64, (3), activation='relu', input_shape=im_shape)(pool2)
    conv3_1=BatchNormalization()(conv3_1)
    pool3=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv3_1)
    flatten=Flatten()(pool3)
    dense_end1 = Dense(64, activation='relu')(flatten)
    dense_end2 = Dense(32, activation='relu')(dense_end1)
    main_output = Dense(5, activation='softmax', name='main_output')(dense_end2)
    
    
    model = Model(inputs= inputs_cnn, outputs=main_output)
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics = ['accuracy'])
    
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

    history=model.fit(X_train, y_train,epochs=10,callbacks=callbacks, batch_size=32,validation_data=(X_test,y_test))
    model.load_weights('best_model.h5')
    return(model,history)

In [ ]:
def evaluate_model(history,X_test,y_test,model):
    scores = model.evaluate((X_test),y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    
    print(history)
    fig1, ax_acc = plt.subplots()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Model - Accuracy')
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()
    
    fig2, ax_loss = plt.subplots()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Model- Loss')
    plt.legend(['Training', 'Validation'], loc='upper right')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.show()
    target_names=['0','1','2','3','4']
    
    y_true=[]
    for element in y_test:
        y_true.append(np.argmax(element))
    prediction_proba=model.predict(X_test)
    prediction=np.argmax(prediction_proba,axis=1)
    cnf_matrix = confusion_matrix(y_true, prediction)

#Model evaluation

In [ ]:
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers.normalization import BatchNormalization
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

model,history=network(X_train,y_train,X_test,y_test)

In [ ]:
evaluate_model(history,X_test,y_test,model)
y_pred=model.predict(X_test)

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Reds):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(10, 10))
plot_confusion_matrix(cnf_matrix, classes=['N', 'S', 'V', 'F', 'Q'],normalize=True,
                      title='Confusion matrix, with normalization')
plt.show()

#XGBoost Model

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
X_train,X_test,y_train,y_test=train_test_split(x,Y,test_size=0.33)
print(len(x),len(Y),len(X_train),len(X_test),len(y_train),len(y_test),X_train.shape,y_train.shape)
params={'max_depth':7,'learning_rate':0.1,'n_estimators':500,'objective':'multi:softprob','num_class':5}
d_train=xgb.DMatrix(X_train,label=y_train))
d_test=xgb.DMatrix(X_test)
clf=xgb.train(params,d_train,num_boost_round=100)
test_y_predict=clf.predict(d_test)
test_y=np.asarray([np.argmax(row) for row in test_y_predict])
print('Accuracy':accuracy_score(y-test,test_y)*100)